In [1]:
import pandas as pd
import numpy as np
import pymysql

In [2]:
conn = pymysql.connect(host="localhost", port=3306, db="project", 
                       user="humanda", password="humanda")

In [3]:
cursor = conn.cursor()

In [4]:
sql = "select * from transportation"
sql2 = "select * from dates"


In [7]:
transportation = pd.read_sql_query(sql, conn)
dates = pd.read_sql_query(sql2, conn)

C:\Users\human\AppData\Local\Temp\ipykernel_34936\2068308783.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  transportation = pd.read_sql_query(sql, conn)
C:\Users\human\AppData\Local\Temp\ipykernel_34936\2068308783.py:2: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  dates = pd.read_sql_query(sql2, conn)


In [8]:
print(transportation.info())
print(dates.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 213374 entries, 0 to 213373
Data columns (total 9 columns):
 #   Column               Non-Null Count   Dtype  
---  ------               --------------   -----  
 0   ID                   213374 non-null  int64  
 1   date                 213374 non-null  object 
 2   transportation_mode  213322 non-null  object 
 3   total_passenger      213374 non-null  int64  
 4   general_passenger    211861 non-null  float64
 5   child_passenger      94532 non-null   float64
 6   youth_passenger      126949 non-null  float64
 7   disabled_passenger   32972 non-null   float64
 8   elderly_passenger    28933 non-null   float64
dtypes: float64(5), int64(2), object(2)
memory usage: 14.7+ MB
None
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1035 entries, 0 to 1034
Data columns (total 3 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   ID                  1035 non-null   int64 
 1   date  

In [10]:
transportation['transportation_mode'] = transportation['transportation_mode'].str.replace(r'[^가-힣]', '', regex=True).str.replace(' ', '')
transportation['date'] = pd.to_datetime(transportation['date'])

In [13]:
bus_only_df = transportation[transportation['transportation_mode'] == "버스"]
train_only_df = transportation[transportation['transportation_mode'] == "지하철"]

In [15]:
dates['date'] = pd.to_datetime(dates['date'])

In [26]:
bus_merged = pd.merge(bus_only_df, dates, on = 'date', how = 'inner')
train_merged = pd.merge(train_only_df, dates, on = 'date', how = 'inner')

In [27]:
bus_merged = bus_merged.drop(columns=['ID_y'])
train_merged = train_merged.drop(columns=['ID_y'])